In [1]:
# -*- coding: utf-8 -*-
#연습문제 : 조달청의 나라장터 입찰 정보 크롤링하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import time
import sys
import re
import math
import random
import os
import pandas as pd
import xlwt


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.

print("\n")
print("=" *100)
query_txt = input('1.공고명으로 검색할 키워드는 무엇입니까?: ')

start_date = input('2.조회 시작일자 입력(예:2019/01/01):')
end_date = input('3.조회 종료일자 입력(예:2019/03/31):')

f_dir=input('4.파일로 저장할 폴더 이름을 쓰세요(예:c:\data\): ')

print("\n")
print("데이터를 수집 중이오니 잠시만 더 기다려 주세요~~^^")
print("\n")

# 저장될 파일위치와 이름을 지정합니다
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

#ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('http://www.g2b.go.kr/')
time.sleep(5) # 이 시간동안 팝업창을 수동으로 닫는다

#driver.switch_to.frame('maintop_iframe') #상단의 통합검색 사용시 이 줄 사용
    
element = driver.find_element_by_id("bidNm")

element.send_keys(query_txt)

#Step 6. 기간 버튼 클릭하여 날짜 입력하기

# 직접 입력 시작 날짜 클릭
s_date = driver.find_element_by_id("fromBidDt")     
driver.find_element_by_id("fromBidDt").click()
s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)

# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.3)

e_date = driver.find_element_by_id("toBidDt")
driver.find_element_by_id("toBidDt").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.3)

#검색 버튼을 누릅니다
search_btn = driver.find_element_by_xpath("""//*[@id="searchForm"]/div/fieldset[1]/ul/li[4]/dl/dd[3]/a/strong""")
search_btn.click()


time.sleep(2)

# 공고 상세 내역 뽑기

count = 1

# 각 항목을 저장할 리스트 생성
upmu2=[]     #업무 
no2=[]       #공고번호-차수
bun2=[]      #분류
gname2=[]    #공고명
url_list2=[] #공고 URL 주소
ggigwan2=[]   #공고기관
sgigwan2=[]   #수요기관
gmethod2=[]  #계약방법
idate2=[]    #입력일시
edate2=[]    #입찰마감일시
gongdong2=[] #공동수급
tuchal2=[]   #투찰
cfile2=[]    #첨부파일유무


print("\n")
#print("현재 페이지의 게시물 정보 표시 ===========================================")

driver.switch_to.frame('sub')
driver.switch_to.frame('main')

# 각 공고별 URL 리스트 출력

for x in range(1,10000) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    print("\n")
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    list_1 = soup.find_all('div','results')
    list_2 = list_1[0].select('table > tbody > tr')
        
    for i in list_2 : 
        print("%s번째 공고내용을 추출합니다~~~~" %count)
            
        all = i.find_all('td')
        
        # 업무 구분
        upmu = all[0].get_text()
        upmu2.append(upmu)
        print("1.업무:" , upmu)
        
        #2. 공고번호-차수
        no=all[1].get_text()
        no2.append(no)
        print("2.공고번호-차수:" , no)
        
        #3. 분류
        bun=all[2].get_text()
        bun2.append(bun)
        print("3.분류:" , bun)
        
         #4. 공고명
        gname=all[3].get_text()
        gname2.append(gname)
        print("4.공고명:" , gname)
        
        #5.공고 URL 주소
        list_3 = i.find('a')['href']
        url_list2.append(list_3)
        print('5.URL 주소:',list_3)
        
        #6.공고 기관
        ggigwan=all[4].get_text()
        ggigwan2.append(ggigwan)
        print('6.공고기관:' , ggigwan)
        
        #7.수요 기관
        sgigwan=all[5].get_text()
        sgigwan2.append(sgigwan)
        print('7.수요기관:',sgigwan)
        
        #8.계약 방법
        gmethod=all[6].get_text()
        gmethod2.append(gmethod)
        print('8.계약방법:',gmethod)
        
        #9.입력 일시
        idate=all[7].get_text()
        idate2.append(idate)
        print('9.입력일시(입찰마감일시):', idate)
        
        #10.입찰마감일시
        #edate=all[8].get_text()
        #edate2.append(edate)
        #print('10.마감일시:',edate)
        
        #11.공동 수급
        try :
            gongdong=all[8].get_text()
        except :
            gongdong=''
        gongdong2.append(gongdong)
        print('11.공동수급:',gongdong)
        
        #12.투찰여부      
        tuchal=all[9].get_text()
        tuchal2.append(tuchal)
        print('12.투찰여부:',tuchal)
        print("\n")
        
        time.sleep(1)
        
        count += 1
        
    try:        
        driver.find_element_by_link_text('''+ 더보기''').click()
    except :
        break
   
    time.sleep(2)

        
print("총 추출 완료한 건수는 %s 건 입니다=========================" %count)
print("\n")

#Step 4. xls 형태와 csv 형태로 저장하기

korea_market = pd.DataFrame()
korea_market['업무']=upmu2
korea_market['공고번호-차수']=pd.Series(no2)
korea_market['분류']=pd.Series(bun2)
korea_market['공고명']=pd.Series(gname2)
korea_market['공고URL주소']=pd.Series(url_list2)
korea_market['공고기관']=pd.Series(ggigwan2)
korea_market['수요기관']=pd.Series(sgigwan2)
korea_market['계약방법']=pd.Series(gmethod2)
korea_market['입력일시(입찰마감일시)']=pd.Series(idate2)
#korea_market['입찰마감일시']=pd.Series(edate2)
korea_market['공동수급']=pd.Series(gongdong2)
korea_market['투찰']=pd.Series(tuchal2)


# csv 형태로 저장하기
korea_market.to_csv(fc_name,encoding="utf-8-sig")

# 엑셀 형태로 저장하기
korea_market.to_excel(fx_name)

e_time = time.time( )
t_time = e_time - s_time

#Step 5. 요약 정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 추출완료 건수는 %s 건 입니다" %count)
#print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )





1.공고명으로 검색할 키워드는 무엇입니까?: 캠프
2.조회 시작일자 입력(예:2019/01/01):2019/01/01
3.조회 종료일자 입력(예:2019/03/31):2019/04/20
4.파일로 저장할 폴더 이름을 쓰세요(예:c:\data\): c:\data\


데이터를 수집 중이오니 잠시만 더 기다려 주세요~~^^




1 페이지 내용 수집 시작합니다 =======================


1번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190426611-00
3.분류: 일반
4.공고명: 2019학년도 글로벌비전 진로캠프(미국서부)해외연수 위탁용역 2단계입찰(규격가격동시) 공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190426611&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 광주시 호남삼육중학교
7.수요기관: 광주시 호남삼육중학교
8.계약방법: 제한(총액)규격가격동시
9.입력일시(입찰마감일시): 2019/04/19 15:50(2019/04/30 10:00)
11.공동수급: 
12.투찰여부: 지문투찰


2번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190427413-00
3.분류: 일반
4.공고명: 2019학년도 호남삼육중『글로벌 비전 진로캠프(미국 동부)』위탁용역 2단계입찰공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190427413&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 광주시 호남삼육중학교
7.수요기관: 광주시 호남삼육중학교
8.계약방법: 제한(총액)규격가격동시
9.입력일시(입찰마감일시): 2019/04/19 15:46(2019/04/30 10:00)
11.공동수급: 
12.투찰여부: 지문투찰


3번째 공고내

23번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190426632-00
3.분류: 일반
4.공고명: 2019학년도 일신여자상업고등학교 글로벌금융캠프 용역업체 입찰 공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190426632&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 서울특별시교육청 일신여자상업고등학교
7.수요기관: 서울특별시교육청 일신여자상업고등학교
8.계약방법: 제한(총액)규격가격동시
9.입력일시(입찰마감일시): 2019/04/16 12:32(2019/04/23 15:00)
11.공동수급: 
12.투찰여부: 지문투찰


24번째 공고내용을 추출합니다~~~~
1.업무: 민간
2.공고번호-차수: 20190426457-00
3.분류: 일반
4.공고명: 2019년도 MBC 1318 사랑의 열매 캠프 납품업체 입찰 공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190426457&bidseq=00&releaseYn=Y&taskClCd=20
6.공고기관: 인천기독교청년회
7.수요기관: 인천기독교청년회
8.계약방법: 일반경쟁
9.입력일시(입찰마감일시): 2019/04/16 11:18(2019/04/30 10:00)
11.공동수급: 
12.투찰여부: 지문투찰


25번째 공고내용을 추출합니다~~~~
1.업무: 민간
2.공고번호-차수: 20190420115-01
3.분류: 취소
4.공고명: 2019년도 MBC 1318 사랑의 열매 캠프 납품업체 입찰 공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190420115&bidseq=01&releaseYn=Y&taskClCd=20
6.공고기관: 인천기독교청년회
7.수요기

46번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190414294-01
3.분류: 변경
4.공고명: 2019학년도 1학년 수련활동 및 리더십캠프
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190414294&bidseq=01&releaseYn=Y&taskClCd=5 
6.공고기관: 인천광역시교육청 인천세무고등학교
7.수요기관: 인천광역시교육청 인천세무고등학교
8.계약방법: 제한(총액)규격가격동시
9.입력일시(입찰마감일시): 2019/04/09 14:36(2019/04/18 16:00)
11.공동수급: 
12.투찰여부: 마감


47번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190414940-00
3.분류: 일반
4.공고명: 2019년 제주특별자치도 대학핵심역량강화지원사업 취업역량강화캠프 프로그램 위탁운영
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190414940&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 제주관광대학
7.수요기관: 제주관광대학
8.계약방법: 일반(총액)협상에의한계약
9.입력일시(입찰마감일시): 2019/04/09 14:00(-)
11.공동수급: 
12.투찰여부: 


48번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190413236-00
3.분류: 일반
4.공고명: 2019학년도 인헌중학교 1학년 진로캠프(수련활동) 위탁용역 입찰공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190413236&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 서울특별시동작교육청 인헌중학교
7.수요기관: 서울특별시동작교육청 

69번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190347924-00
3.분류: 긴급
4.공고명: 서울외고 2학년 리더십캠프 용역 업체 선정
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190347924&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 서울특별시교육청 서울외국어고등학교
7.수요기관: 서울특별시교육청 서울외국어고등학교
8.계약방법: 제한(총액)2단계경쟁
9.입력일시(입찰마감일시): 2019/03/28 15:35(2019/04/04 12:00)
11.공동수급: 
12.투찰여부: 마감


70번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190343312-00
3.분류: 일반
4.공고명: 어린이 안전문화 체험캠프 운영 용역(재공고)
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190343312&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 충청북도 청주시
7.수요기관: 충청북도 청주시
8.계약방법: 수의(총액)소액-견적입찰(2인 이상 견적 제출)
9.입력일시(입찰마감일시): 2019/03/26 15:44(2019/04/01 10:00)
11.공동수급: 
12.투찰여부: 마감


8 페이지 내용 수집 시작합니다 =======================


71번째 공고내용을 추출합니다~~~~
1.업무: 공사
2.공고번호-차수: 20190342376-00
3.분류: 일반
4.공고명: 캠프라과디아 내 도시계획도로(중로3-3호선 외 3개소) 개설사업
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190342376&bidseq=00&releaseYn=Y&ta

92번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190327008-00
3.분류: 일반
4.공고명: 2019년 이목중학교 진로 어울림 캠프 수련활동 위탁용역 입찰 공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190327008&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 경기도수원교육청 이목중학교
7.수요기관: 경기도수원교육청 이목중학교
8.계약방법: 제한(총액)규격가격동시
9.입력일시(입찰마감일시): 2019/03/18 14:09(2019/03/26 16:00)
11.공동수급: 
12.투찰여부: 마감


93번째 공고내용을 추출합니다~~~~
1.업무: 공사
2.공고번호-차수: 20190326310-00
3.분류: 일반
4.공고명: 캠프워커 교차로 조경지 정비공사
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190326310&bidseq=00&releaseYn=Y&taskClCd=3 
6.공고기관: 대구광역시 남구
7.수요기관: 대구광역시 남구
8.계약방법: 수의(소액)-견적입찰(2인 이상 견적 제출)
9.입력일시(입찰마감일시): 2019/03/18 11:18(2019/03/22 12:00)
11.공동수급: 
12.투찰여부: 마감


94번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190320859-00
3.분류: 일반
4.공고명: [IITP]2019학년도 대덕소프트웨어마이스터고등학교 취업역량강화캠프 위탁용역 2단계 입찰 공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190320859&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 대전광역시교육청 대

115번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190234960-00
3.분류: 일반
4.공고명: 사이버 가디언즈 보안캠프 운영
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190234960&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 한국정보기술연구원                                
7.수요기관: 한국정보기술연구원                                
8.계약방법: 일반(총액)협상에의한계약
9.입력일시(입찰마감일시): 2019/02/26 17:31(-)
11.공동수급: 
12.투찰여부: 


116번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190231926-00
3.분류: 일반
4.공고명: 2019년 청원 힐링캠프 위탁용역
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190231926&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 전라북도 전주시
7.수요기관: 전라북도 전주시
8.계약방법: 일반(총액)협상에의한계약
9.입력일시(입찰마감일시): 2019/02/25 15:03(-)
11.공동수급: 
12.투찰여부: 


117번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190231231-00
3.분류: 긴급
4.공고명: 2019년 통일리더캠프(국내) 위탁 용역
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190231231&bidseq=00&releaseYn=Y&taskClCd=1 
6.공고기관: 조달청 서울지방조달청
7.수요기관: 통일부 통일교육원
8.계약방법: 일반(총액)협상에의

138번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190200865-00
3.분류: 일반
4.공고명: 부산공업고등학교 도제역량강화캠프 위탁용역 소액수의 견적제출 안내공고
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190200865&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 부산광역시교육청 부산공업고등학교
7.수요기관: 부산광역시교육청 부산공업고등학교
8.계약방법: 수의(총액)소액수의
9.입력일시(입찰마감일시): 2019/02/01 14:29(2019/02/12 10:00)
11.공동수급: 
12.투찰여부: 마감


139번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190200141-00
3.분류: 일반
4.공고명: 서울창의마을 풍납캠프 활용 기본계획 수립 연구 용역
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190200141&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 서울특별시
7.수요기관: 서울특별시
8.계약방법: 제한(총액)협상에의한계약
9.입력일시(입찰마감일시): 2019/02/01 09:57(2019/02/15 16:00)
11.공동수급: 마감
12.투찰여부: 마감


140번째 공고내용을 추출합니다~~~~
1.업무: 공사
2.공고번호-차수: 20190133282-00
3.분류: 일반
4.공고명: 양평 맑은숲캠프 야외 물놀이시설 조성공사
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190133282&bidseq=00&releaseYn=Y&taskClCd=3 
6.공고기관: 조달청 서울지방조달청
7.수요기관: 양평공사
8.계약방법: 지역제한


17 페이지 내용 수집 시작합니다 =======================


161번째 공고내용을 추출합니다~~~~
1.업무: 용역
2.공고번호-차수: 20190101597-00
3.분류: 일반
4.공고명: 2018학년도 글로벌 역사문화탐구캠프 위탁용역
5.URL 주소: http://www.g2b.go.kr:8081/ep/invitation/publish/bidInfoDtl.do?bidno=20190101597&bidseq=00&releaseYn=Y&taskClCd=5 
6.공고기관: 전라남도교육청 해남고등학교
7.수요기관: 전라남도교육청 해남고등학교
8.계약방법: 수의(총액)소액수의
9.입력일시(입찰마감일시): 2019/01/03 11:53(2019/01/09 10:00)
11.공동수급: 
12.투찰여부: 마감


총 추출 완료한 건수는 162 건 입니다=========================




총 소요시간은 244.3 초 입니다 
총 추출완료 건수는 162 건 입니다
파일 저장 완료: csv 파일명 : c:\data\2019-04-22-10-03-34-캠프\2019-04-22-10-03-34-캠프.csv 
파일 저장 완료: xls 파일명 : c:\data\2019-04-22-10-03-34-캠프\2019-04-22-10-03-34-캠프.xls 
